In [1]:
import os
print(os.getcwd())

import warnings
warnings.filterwarnings('ignore') 

/home/piyushsamant/Data/MicroRNA cohort data by Jin u for Generative AI/Experiment with Ridge Model small miRNA set


In [10]:
import pandas as pd
import numpy as np
import random
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.metrics import roc_curve, auc, roc_auc_score, make_scorer, accuracy_score
from sklearn.linear_model import SGDClassifier, ElasticNetCV, RidgeClassifier
from sklearn.model_selection import GridSearchCV, StratifiedKFold, train_test_split, KFold, cross_val_score
from sklearn.datasets import make_classification
from keras.initializers import RandomNormal
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.utils import shuffle

from tensorflow.keras.layers import Input, Dense, Reshape, Flatten, Dropout, multiply, Embedding, BatchNormalization, LeakyReLU
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.optimizers import Adam
from keras import backend
from keras.backend import mean
from keras.optimizers import SGD




In [63]:
# Defineing the CGan Model and training function in a single class


import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.utils import shuffle
from tensorflow.keras.layers import Input, Dense, Reshape, Flatten, Dropout, multiply, Embedding, BatchNormalization, LeakyReLU
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.optimizers import Adam
from keras.backend import mean
from keras.optimizers import SGD
from keras import backend


def wasserstein_loss(y_true, y_pred):
    return mean(y_true * y_pred)


class cGAN1():
    def __init__(self, latent_dim = 32, out_shape = 24):
        self.latent_dim = latent_dim
        self.out_shape = out_shape
        self.num_classes = 2
        # using Adam as our optimizer

        # Experiment with optimizer, learning rate, Beta1 
        # optimizer = Adam(0.00001, 0.5)
        optimizer = Adam(0.00001, 0.9)
        # optimizer = SGD(lr = 0.0001, momentum = 0.5, nesterov = True)

        # building the discriminator
        self.discriminator = self.discriminator()
        
        # self.discriminator.compile(loss=['binary_crossentropy'], optimizer = optimizer,metrics=['accuracy'])
        self.discriminator.compile(loss = wasserstein_loss, optimizer = optimizer, metrics = ['accuracy'])

        # building the generator
        self.generator = self.generator()

        noise = Input(shape=(self.latent_dim,))
        label = Input(shape=(1,))
        gen_samples = self.generator([noise, label])

        # we don't train discriminator when training generator
        self.discriminator.trainable = False
        valid = self.discriminator([gen_samples, label])

        # combining both models
        self.combined = Model([noise, label], valid)
        
        # self.combined.compile(loss=['binary_crossentropy'], optimizer=optimizer, metrics=['accuracy'])
        self.combined.compile(loss = wasserstein_loss, optimizer = optimizer, metrics = ['accuracy'])


        self.G_losses = []
        self.D_losses = []

    def generator(self):
        init = RandomNormal(mean=0.0, stddev=0.02)
        model = Sequential()

        model.add(Dense(128, input_dim=self.latent_dim))
        model.add(Dropout(0.2))
        model.add(LeakyReLU(alpha=0.2))
        model.add(BatchNormalization(momentum=0.8))

        model.add(Dense(256))
        model.add(Dropout(0.2))
        model.add(LeakyReLU(alpha=0.2))
        model.add(BatchNormalization(momentum=0.8))

        model.add(Dense(512))
        model.add(Dropout(0.2))
        model.add(LeakyReLU(alpha=0.2))
        model.add(BatchNormalization(momentum=0.8))

        model.add(Dense(self.out_shape, activation='tanh'))

        noise = Input(shape=(self.latent_dim,))
        label = Input(shape=(1,), dtype='int32')
        label_embedding = Flatten()(Embedding(self.num_classes, self.latent_dim)(label))

        model_input = multiply([noise, label_embedding])
        gen_sample = model(model_input)

        return Model([noise, label], gen_sample, name="Generator")

    def discriminator(self):
        init = RandomNormal(mean=0.0, stddev=0.02)
        model = Sequential()

        model.add(Dense(512, input_dim=self.out_shape, kernel_initializer=init))
        model.add(LeakyReLU(alpha=0.2))

        model.add(Dense(256, kernel_initializer=init))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dropout(0.4))

        model.add(Dense(128, kernel_initializer=init))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dropout(0.4))

        model.add(Dense(1, activation = 'sigmoid'))

        gen_sample = Input(shape=(self.out_shape,))
        label = Input(shape=(1,), dtype = 'int32')
        label_embedding = Flatten()(Embedding(self.num_classes, self.out_shape)(label))

        model_input = multiply([gen_sample, label_embedding])
        validity = model(model_input)

        return Model(inputs=[gen_sample, label], outputs=validity, name = "Discriminator")

    def train(self, X_train, y_train, pos_index, neg_index, epochs, sampling=False, batch_size = 32, sample_interval = 100, plot=True):
        # Adversarial ground truths
        valid = np.ones((batch_size, 1))
        fake = np.zeros((batch_size, 1))

        for epoch in range(epochs):
            if sampling:
                idx1 = np.random.choice(pos_index, 8)
                idx0 = np.random.choice(neg_index, batch_size - 8)
                idx = np.concatenate((idx1, idx0))
            else:
                idx = np.random.choice(len(y_train), batch_size)
            samples, labels = X_train[idx], y_train[idx]
            samples, labels = shuffle(samples, labels)

            # Sample noise as generator input
            noise = np.random.normal(0, 1, (batch_size, self.latent_dim))
            gen_samples = self.generator.predict([noise, labels])

            if epoch < epochs // 1.5:
                valid_smooth = (valid + 0.1) - (np.random.random(valid.shape) * 0.1)
                fake_smooth = (fake - 0.1) + (np.random.random(fake.shape) * 0.1)
            else:
                valid_smooth = valid
                fake_smooth = fake

            # Train the discriminator
            self.discriminator.trainable = True
            d_loss_real = self.discriminator.train_on_batch([samples, labels], valid_smooth)
            d_loss_fake = self.discriminator.train_on_batch([gen_samples, labels], fake_smooth)
            d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)

            # Train Generator
            self.discriminator.trainable = False
            sampled_labels = np.random.randint(0, 2, batch_size).reshape(-1, 1)
            g_loss = self.combined.train_on_batch([noise, sampled_labels], valid)

            if (epoch + 1) % sample_interval == 0:
                print('[%d/%d]\tLoss_D: %.4f\tLoss_G: %.4f' % (epoch, epochs, d_loss[0], g_loss[0]))
            self.G_losses.append(g_loss[0])
            self.D_losses.append(d_loss[0])

            if plot:
                if epoch + 1 == epochs:
                    plt.figure(figsize=(3, 2))
                    plt.title("Generator and Discriminator Loss")
                    plt.plot(self.G_losses, label="G")
                    plt.plot(self.D_losses, label="D")
                    plt.xlabel("iterations")
                    plt.ylabel("Loss")
                    plt.legend()
                    plt.show()


In [ ]:
# No of miRNAs need to be consider
no_of_miRNA = 24
# no of epoches in the experiment
iteration = 1500

# Calling C Gan in actial code 
cgan = cGAN1(out_shape = no_of_miRNA)

y_outer_train_1 = y_train.reshape(-1,1)  # X_train and y_train are ndarray of feature and target of the training  
pos_index = np.where( y_train_1 == 1)[0]
neg_index = np.where( y_train_1 == 0)[0]
cgan.train(X_train, y_train, pos_index, neg_index, epochs = iteration) # iteration is typically more than 1000 but can change accordingly

In [ ]:
# Once Model is trained and we have obesrved the G and D Losses can generate new datapoints

# Defining a random noise according to the no of new samples generated
noise = np.random.normal(0, 1, (1000, 32))
        
# To generate 500 instances with class value 0 
sampled_labels = np.zeros(1000).reshape(-1, 1)
gen_samples = cgan.generator.predict([noise, sampled_labels])
gen_df_zeros = pd.DataFrame(data = gen_samples, columns = X_pd.columns)  # X_pd is the input dataframe with mirnas as columns
gen_df_zeros['Class'] = 0
        
# To generate 500 instances with class value 1 
sampled_labels_2 = np.ones(1000).reshape(-1, 1)
gen_samples_2 = cgan.generator.predict([noise, sampled_labels_2])
gen_df_ones = pd.DataFrame(data = gen_samples_2, columns = X_pd.columns)
gen_df_ones['Class'] = 1


# Combining the generated datasets for zero and ones

combine_df_generated = pd.concat([gen_df_ones, gen_df_zeros], ignore_index=True, sort=False)
combine_df_generated = combine_df_generated.sample(frac=1).reset_index(drop=True)

X_generated = combine_df_generated.drop('Class', axis = 1)
y_generated = combine_df_generated['Class']
